Importar librerias

In [11]:
import pandas as pd
import numpy as np
from datetime import datetime, date

Concatenar nuevamente los df de ratings

In [43]:
df_rating1 = pd.read_csv('../P1_ML/MLOpsReviews/ratings/1.csv', sep = ',')
df_rating2 = pd.read_csv('../P1_ML/MLOpsReviews/ratings/2.csv', sep = ',')
df_rating3 = pd.read_csv('../P1_ML/MLOpsReviews/ratings/3.csv', sep = ',')
df_rating4 = pd.read_csv('../P1_ML/MLOpsReviews/ratings/4.csv', sep = ',')
df_rating5 = pd.read_csv('../P1_ML/MLOpsReviews/ratings/5.csv', sep = ',')
df_rating6 = pd.read_csv('../P1_ML/MLOpsReviews/ratings/6.csv', sep = ',')
df_rating7 = pd.read_csv('../P1_ML/MLOpsReviews/ratings/7.csv', sep = ',')
df_rating8 = pd.read_csv('../P1_ML/MLOpsReviews/ratings/8.csv', sep = ',')

In [44]:
for ind, time in enumerate(df_rating1.timestamp):
    df_rating1['timestamp'][ind] = date.fromtimestamp(time)
for ind, time in enumerate(df_rating2.timestamp):
    df_rating2['timestamp'][ind] = date.fromtimestamp(time)
for ind, time in enumerate(df_rating3.timestamp):
    df_rating3['timestamp'][ind] = date.fromtimestamp(time)
for ind, time in enumerate(df_rating4.timestamp):
    df_rating4['timestamp'][ind] = date.fromtimestamp(time)
for ind, time in enumerate(df_rating5.timestamp):
    df_rating5['timestamp'][ind] = date.fromtimestamp(time)
for ind, time in enumerate(df_rating6.timestamp):
    df_rating6['timestamp'][ind] = date.fromtimestamp(time)
for ind, time in enumerate(df_rating7.timestamp):
    df_rating7['timestamp'][ind] = date.fromtimestamp(time)
for ind, time in enumerate(df_rating8.timestamp):
    df_rating8['timestamp'][ind] = date.fromtimestamp(time)

C:\Users\Carlo\AppData\Local\Temp\ipykernel_15892\4204255159.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rating1['timestamp'][ind] = date.fromtimestamp(time)
C:\Users\Carlo\AppData\Local\Temp\ipykernel_15892\4204255159.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rating2['timestamp'][ind] = date.fromtimestamp(time)
C:\Users\Carlo\AppData\Local\Temp\ipykernel_15892\4204255159.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

In [45]:
df_ratings = pd.concat([df_rating1, df_rating2, df_rating3, df_rating4, df_rating5, df_rating6, df_rating7, df_rating8], ignore_index=True).drop_duplicates()

In [48]:
df_movies = pd.read_csv('Ratings', sep = ';')